1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других

In [1]:
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.metrics import roc_auc_score, log_loss, confusion_matrix

In [2]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    

In [3]:
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    

In [4]:
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [5]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [6]:
def f_score(y_true, y_pred, b=1):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    fscore = (1 + b ** 2) * (precision * recall) / (b ** 2 * precision + recall)
    r_a_score = roc_auc_score(y_true, y_pred)
    np.nan_to_num(fscore, copy=False)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    return thresholds[ix], fscore[ix], precision[ix], recall[ix], r_a_score
    

___

In [7]:
df = pd.read_csv('train_case2.csv', delimiter=';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [8]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [9]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [10]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификатор и запустим кросс-валидацию

In [11]:
model_list = [LogisticRegression, 
              RandomForestClassifier, 
              GradientBoostingClassifier, 
              LGBMClassifier]
classifier_list = [make_pipeline(feats, model(random_state = 42)) for model in model_list]

In [12]:
for pipe in classifier_list:
    cv_scores = cross_val_score(pipe, X_train, y_train, cv=15, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    print(f'for \033[1m{pipe.steps[1][0]}\033[0m CV score is {cv_score:.3f} \u00b1 {cv_score_std:.3f}')

for logisticregression CV score is 0.787 ± 0.008
for randomforestclassifier CV score is 0.773 ± 0.006
for gradientboostingclassifier CV score is 0.803 ± 0.008
for lgbmclassifier CV score is 0.802 ± 0.007


In [13]:
names = []
metrics = []
for pipe in classifier_list:
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict_proba(X_test)[:, 1]
    names.append(pipe.steps[1][0])
    thr, f1, pr, rec, roc = f_score(y_test.values, y_pred, b=1)
    tn, fp, fn, tp = confusion_matrix(y_test.values, y_pred > thr).ravel()
    metrics.append([thr, f1, pr, rec, roc, tp, tn, fp, fn])

In [14]:
result_df = pd.DataFrame(metrics, 
                         index=names, 
                         columns=['Threshhold', 'F-score', 'Precision', 'Recall', 'ROC-AUC', 'TP', 'TN', 'FP', 'FN'])
result_df['FPR'] = result_df['FP'] / (result_df['FP'] + result_df['TN'])

In [15]:
result_df

,Threshhold,F-score,Precision,Recall,ROC-AUC,TP,TN,FP,FN,FPR
logisticregression,0.386937,0.730323,0.647431,0.837558,0.784035,7269,4861,3959,1411,0.448866
randomforestclassifier,0.350000,0.718863,0.642669,0.815553,0.771037,7017,4991,3829,1663,0.434127
gradientboostingclassifier,0.394947,0.740248,0.697848,0.788134,0.802615,6840,5858,2962,1840,0.335828
lgbmclassifier,0.344797,0.739561,0.660803,0.839631,0.801689,7287,5079,3741,1393,0.424150


При b = 1(То есть в случае, когда для нас равно важны и precision, и recall)  

Можно сделать вывод, что в нашем случае лучший результат(при условии базовых настроек моделей) показывает модель GradientBoostingClassifier из паката sklearn. FPR удалось снизить на 11% по отношению к рассматриваемой на занятии логистической регрессии. Кроме того, несмотря на в целом меньшее количество TP, значение precision также выше.  

Далее нужно уже смотреть, что для нас важнее: охватить всех возможных больных или меньшее их количество, но с большей точностью.

___

5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

___

Осями на графике ROC являются значения TPR/FPR  
На PR - precision и recall соответственно

$$ TPR = recall = \frac{TP}{TP+FN} $$  

$$ FPR = \frac{FP}{FP+TN} $$  

$$ precision = \frac{TP}{TP+FP} $$  

Очевидно, что оси TPR и recall одинаковы, поэтому разницу нужно искать между FPR и precision

Можно провести эксперимент и посчитать соответствующие метрики для каждой из моделей

In [16]:
# Количество наблюдений
N = 100000

# Модель 1
TP1 = 90
FP1 = 10
FN1 = 10
TN1 = 99890

# Модель 2
TP2 = 90
FP2 = 910
FN2 = 10
TN2 = 98990

In [17]:
pr = lambda tp, fp: tp / (tp + fp)
fpr = lambda fp, tn: fp / (fp + tn)

In [18]:
print(f'precision M1: {pr(TP1, FP1)}\nprecision M2: {pr(TP2, FP2)}')

precision M1: 0.9
precision M2: 0.09


In [19]:
print(f'FPR M1: {fpr(FP1, TN1):.5f}\nFPR M2: {fpr(FP2, TN2):.5f}')

FPR M1: 0.00010
FPR M2: 0.00911


Очевидно, что значение FPR меняется от модели к модели незначительно, а вот precision отличается серьезно, что неудивительно, так как в нем учитывается количество ложно-положительных результатов. 

Можно сделать вывод, что для дисбалансных выборок PR-AUC метрика подходит лучше